In [1]:
# import basic python libraries
import numpy as np
import pandas as pd

# load sequence data
sequence_data = pd.read_csv('Waltz_and_AAIndex1_Data_Filtered_Best_Features_And_Waltz_Features')

training_data = sequence_data.drop(['Classification'], axis = 1)
target_data = sequence_data['Classification']

In [2]:
# import data pre-processing package
from sklearn.model_selection import train_test_split

# Split the dataset in two training and test sets: use an 80:20 split
X_train, X_test, y_train, y_test = train_test_split(training_data, target_data, test_size = 0.2)

In [3]:
chosen_Best =  ['VENT840101_mean', 'ROBB760106_mean',  'BASU050102_mean', 'GUYH850101_mean',
                'CASG920101_mean', 'CORJ870108_mean',  'KRIW790101_mean', 'CORJ870101_mean',  
                'BULH740101_mean', 'PONP800101_mean',  'KANM800102_mean', 'ZHOH040103_mean',  
                'ZHOH040101_mean', 'ENGD860101_range', 'FUKS010102_mean', 'RACS770103_mean',  
                'MIYS990104_mean', 'CORJ870103_mean',  'GEIM800107_mean', 'LEVM780106_mean', 
               ]

chosen_Waltz = ['pos3_NOZY710101', 'pos4_NOZY710101',  'pos4_VASM830103', 'pos1_PALJ810104', 
                'pos2_PALJ810104', 'pos0_CHOP780206',  'pos1_ROBB760102', 'pos0_GEIM800107', 
                'pos1_GEIM800107', 'pos2_GEIM800107',  'pos2_GARJ730101', 'pos0_FAUJ880110', 
                'pos1_FAUJ880110', 'pos2_FAUJ880110',  'pos3_FAUJ880110', 'pos4_FAUJ880110',
                'pos0_VENT840101', 'pos0_RACS820114',  'pos2_RACS820114', 'pos4_RACS820114', 
                'pos3_ONEK900102', 'pos4_ONEK900102',  'pos5_ONEK900102', 'pos1_PTIO830102', 
                'pos2_PTIO830102', 'pos5_FINA910102',  'pos5_MAXF760104', 'pos0_ZIMJ680103', 
                'pos5_ZIMJ680103', 'pos1_QIAN880123',  'pos5_AURR980106', 'pos0_FINA910102', 
                'pos1_FINA910102', 'pos2_FINA910102',  'pos3_FINA910102', 'pos4_FINA910102'
                ]

In [4]:
positional_features = []
for i in range(0, 6):
    for j in range(0, 20):
        positional_features.append('pos' + str(i) + '_orth_' + str(j))

In [5]:
X_train_Best = X_train[positional_features + chosen_Best]
X_test_Best = X_test[positional_features + chosen_Best]

X_train_Waltz = X_train[positional_features + chosen_Waltz]
X_test_Waltz = X_test[positional_features + chosen_Waltz]

In [6]:
# list the four machine learning methods to be used
methods = ['svm', 'forest', 'logistic', 'mlp']
methods_reduced = ['forest', 'logistic', 'mlp']

In [7]:
# define a dictionary of parameters for each machine learning method
parameters_Best =  {'svm' :      {'kernel': ['linear', 'rbf'],
                                  'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000], # error term penalty parameter
                                  'gamma': [0.001, 0.01, 0.1]}, # kernel coefficient (rbf only)
              
                    'forest' :   {'n_estimators': [10, 100, 1000], # number of decision trees in forest
                                  'max_depth': [1, 10, 100, 1000]}, # maximum tree depth
              
                    'logistic' : {'C' : [0.001, 0.01, 0.1, 1, 10, 100, 1000]}, # inverse of regularization strength
              
                    'mlp' :      {'activation' : ['identity', 'logistic', 'tanh'], # mlp activation function
                                  'hidden_layer_sizes' : [(10, 1), (20, 1), (30, 1), (50, 1)], # size of single hidden layer
                                  'learning_rate_init' : [0.1, 0.01, 0.001, 0.0001, 0.00001]} # training rate
             }

parameters_Waltz = {'svm' :      {'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]}, # error term penalty parameter
              
                    'forest' :   {'n_estimators': [10, 100, 1000], # number of decision trees in forest
                                  'max_depth': [1, 10, 100, 1000]}, # maximum tree depth
              
                    'logistic' : {'C' : [0.001, 0.01, 0.1, 1, 10, 100, 1000]}, # inverse of regularization strength
              
                    'mlp' :      {'activation' : ['identity', 'logistic', 'tanh'], # mlp activation function
                                  'hidden_layer_sizes' : [(10, 1), (20, 1), (30, 1), (50, 1)], # size of single hidden layer
                                  'learning_rate_init' : [0.1, 0.01, 0.001, 0.0001, 0.00001]} # training rate
             }

In [8]:
# import grid search package and machine learning libraries
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier

num_folds = 10 # we are doing 10-fold cross validation
perf_metric = 'roc_auc' # use mean area under ROC curve for performance evaluation 

# define dictionary of classifier objects for each machine learning method
classifiers_Best =  {'svm' : GridSearchCV(SVC(probability = True), parameters_Best['svm'], cv = num_folds, scoring = perf_metric),
                     'forest' : GridSearchCV(RandomForestClassifier(), parameters_Best['forest'], cv = num_folds, scoring = perf_metric),
                     'logistic' : GridSearchCV(LogisticRegression(), parameters_Best['logistic'], cv = num_folds, scoring = perf_metric),
                     'mlp' : GridSearchCV(MLPClassifier(max_iter = 1000), parameters_Best['mlp'], cv = num_folds, scoring = perf_metric)
                    }

classifiers_Waltz = {'svm' : GridSearchCV(LinearSVC(), parameters_Waltz['svm'], cv = num_folds, scoring = perf_metric),
                     'forest' : GridSearchCV(RandomForestClassifier(), parameters_Waltz['forest'], cv = num_folds, scoring = perf_metric),
                     'logistic' : GridSearchCV(LogisticRegression(), parameters_Waltz['logistic'], cv = num_folds, scoring = perf_metric),
                     'mlp' : GridSearchCV(MLPClassifier(max_iter = 1000), parameters_Waltz['mlp'], cv = num_folds, scoring = perf_metric)
                    }

In [9]:
# train each method on the training data
for method in methods:
    classifiers_Best[method].fit(X_train_Best, y_train)
    print(method, 'complete for best 20 features')

svm complete for best 20 features
forest complete for best 20 features
logistic complete for best 20 features


C:\Users\Freddie\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.
  % (), ConvergenceWarning)
C:\Users\Freddie\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.
  % (), ConvergenceWarning)
C:\Users\Freddie\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.
  % (), ConvergenceWarning)
C:\Users\Freddie\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.
  % (), ConvergenceWarning)
C:\Users\Freddie\Anaconda3\lib\site-packages\skl

mlp complete for best 20 features


In [10]:
# train each method on the training data
for method in methods:
    classifiers_Waltz[method].fit(X_train_Waltz, y_train)
    print(method, 'complete for Waltz features')

svm complete for Waltz features
forest complete for Waltz features
logistic complete for Waltz features


C:\Users\Freddie\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.
  % (), ConvergenceWarning)
C:\Users\Freddie\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.
  % (), ConvergenceWarning)
C:\Users\Freddie\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.
  % (), ConvergenceWarning)
C:\Users\Freddie\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.
  % (), ConvergenceWarning)
C:\Users\Freddie\Anaconda3\lib\site-packages\skl

mlp complete for Waltz features


In [13]:
# collect results for each grid search into a dictionary
results_Best = {}
for method in methods:
    # mean_test_score is the overall set of results for each 10-fold cross validation
    results_Best.update({method : {'mean_test_score' : classifiers_Best[method].cv_results_['mean_test_score'].tolist()}})
    for parameter in parameters_Best[method]:
        # save results listed by each parameter value
        results_Best[method].update({parameter : classifiers_Best[method].cv_results_['param_' + parameter].compressed().tolist()})

In [14]:
# collect results for each grid search into a dictionary
results_Waltz = {}
for method in methods:
    # mean_test_score is the overall set of results for each 10-fold cross validation
    results_Waltz.update({method : {'mean_test_score' : classifiers_Waltz[method].cv_results_['mean_test_score'].tolist()}})
    for parameter in parameters_Waltz[method]:
        # save results listed by each parameter value
        results_Waltz[method].update({parameter : classifiers_Waltz[method].cv_results_['param_' + parameter].compressed().tolist()})

In [23]:
# zip results together for each method, into a list of tuples, and also add an index entry to identify each parameter
for method in methods:
    results_Best[method]['zip'] = list(zip(*[results_Best[method][x] for x in parameters_Best[method]], results_Best[method]['mean_test_score']))
    results_Best[method]['zip_index'] = list([x for x in parameters_Best[method]])
    results_Waltz[method]['zip'] = list(zip(*[results_Waltz[method][x] for x in parameters_Waltz[method]], results_Waltz[method]['mean_test_score']))
    results_Waltz[method]['zip_index'] = list([x for x in parameters_Waltz[method]])

In [27]:
results = results_Best
parameters = parameters_Best

# define parameters for plotting
kernel_index = results['svm']['zip_index'].index('kernel')
gamma_index = results['svm']['zip_index'].index('gamma')
svm_linear_C_Best = parameters['svm']['C']
svm_linear_results_Best = [x[3] for x in results['svm']['zip'] if x[kernel_index] == 'linear' and x[gamma_index] == 0.1]

rbf_gamma_1_results_Best = [x[3] for x in results['svm']['zip'] if (x[kernel_index] == 'rbf') and (x[gamma_index] == 0.1)]
rbf_gamma_01_results_Best = [x[3] for x in results['svm']['zip'] if (x[kernel_index] == 'rbf') and (x[gamma_index] == 0.01)]
rbf_gamma_001_results_Best = [x[3] for x in results['svm']['zip'] if (x[kernel_index] == 'rbf') and (x[gamma_index] == 0.001)]

depth_index = results['forest']['zip_index'].index('max_depth')
forest_max_depth_1_results_Best = [x[2] for x in results['forest']['zip'] if (x[depth_index] == 1)]
forest_max_depth_10_results_Best = [x[2] for x in results['forest']['zip'] if (x[depth_index] == 10)]
forest_max_depth_100_results_Best = [x[2] for x in results['forest']['zip'] if (x[depth_index] == 100)]
forest_max_depth_1000_results_Best = [x[2] for x in results['forest']['zip'] if (x[depth_index] == 1000)]

forest_n_estimators_Best = parameters['forest']['n_estimators']

logistic_C_Best = parameters['logistic']['C']
logistic_results_Best = [x[1] for x in results['logistic']['zip']]

mlp_learning_rate_init_Best = parameters['mlp']['learning_rate_init']

activation_index = results['mlp']['zip_index'].index('activation')
size_index = results['mlp']['zip_index'].index('hidden_layer_sizes')
mlp_identity_10_results_Best = [x[3] for x in results['mlp']['zip'] if (x[activation_index] == 'identity') and (x[size_index] == (10, 1))]
mlp_identity_20_results_Best = [x[3] for x in results['mlp']['zip'] if (x[activation_index] == 'identity') and (x[size_index] == (20, 1))]
mlp_identity_30_results_Best = [x[3] for x in results['mlp']['zip'] if (x[activation_index] == 'identity') and (x[size_index] == (30, 1))]
mlp_identity_50_results_Best = [x[3] for x in results['mlp']['zip'] if (x[activation_index] == 'identity') and (x[size_index] == (50, 1))]

mlp_logistic_10_results_Best = [x[3] for x in results['mlp']['zip'] if (x[activation_index] == 'logistic') and (x[size_index] == (10, 1))]
mlp_logistic_20_results_Best = [x[3] for x in results['mlp']['zip'] if (x[activation_index] == 'logistic') and (x[size_index] == (20, 1))]
mlp_logistic_30_results_Best = [x[3] for x in results['mlp']['zip'] if (x[activation_index] == 'logistic') and (x[size_index] == (30, 1))]
mlp_logistic_50_results_Best = [x[3] for x in results['mlp']['zip'] if (x[activation_index] == 'logistic') and (x[size_index] == (50, 1))]

mlp_tanh_10_results_Best = [x[3] for x in results['mlp']['zip'] if (x[activation_index] == 'tanh') and (x[size_index] == (10, 1))]
mlp_tanh_20_results_Best = [x[3] for x in results['mlp']['zip'] if (x[activation_index] == 'tanh') and (x[size_index] == (20, 1))]
mlp_tanh_30_results_Best = [x[3] for x in results['mlp']['zip'] if (x[activation_index] == 'tanh') and (x[size_index] == (30, 1))]
mlp_tanh_50_results_Best = [x[3] for x in results['mlp']['zip'] if (x[activation_index] == 'tanh') and (x[size_index] == (50, 1))]

#-------------------------------------------------------------------------

results = results_Waltz
parameters = parameters_Waltz

# define parameters for plotting
svm_linear_C_Waltz = parameters['svm']['C']
svm_linear_results_Waltz = [x[1] for x in results['svm']['zip']]

rbf_gamma_1_results_Waltz = [x[3] for x in results['svm']['zip'] if (x[kernel_index] == 'rbf') and (x[gamma_index] == 0.1)]
rbf_gamma_01_results_Waltz = [x[3] for x in results['svm']['zip'] if (x[kernel_index] == 'rbf') and (x[gamma_index] == 0.01)]
rbf_gamma_001_results_Waltz = [x[3] for x in results['svm']['zip'] if (x[kernel_index] == 'rbf') and (x[gamma_index] == 0.001)]

depth_index = results['forest']['zip_index'].index('max_depth')
forest_max_depth_1_results_Waltz = [x[2] for x in results['forest']['zip'] if (x[depth_index] == 1)]
forest_max_depth_10_results_Waltz = [x[2] for x in results['forest']['zip'] if (x[depth_index] == 10)]
forest_max_depth_100_results_Waltz = [x[2] for x in results['forest']['zip'] if (x[depth_index] == 100)]
forest_max_depth_1000_results_Waltz = [x[2] for x in results['forest']['zip'] if (x[depth_index] == 1000)]

forest_n_estimators_Waltz = parameters['forest']['n_estimators']

logistic_C_Waltz = parameters['logistic']['C']
logistic_results_Waltz = [x[1] for x in results['logistic']['zip']]

mlp_learning_rate_init_Waltz = parameters['mlp']['learning_rate_init']

activation_index = results['mlp']['zip_index'].index('activation')
size_index = results['mlp']['zip_index'].index('hidden_layer_sizes')
mlp_identity_10_results_Waltz = [x[3] for x in results['mlp']['zip'] if (x[activation_index] == 'identity') and (x[size_index] == (10, 1))]
mlp_identity_20_results_Waltz = [x[3] for x in results['mlp']['zip'] if (x[activation_index] == 'identity') and (x[size_index] == (20, 1))]
mlp_identity_30_results_Waltz = [x[3] for x in results['mlp']['zip'] if (x[activation_index] == 'identity') and (x[size_index] == (30, 1))]
mlp_identity_50_results_Waltz = [x[3] for x in results['mlp']['zip'] if (x[activation_index] == 'identity') and (x[size_index] == (50, 1))]

mlp_logistic_10_results_Waltz = [x[3] for x in results['mlp']['zip'] if (x[activation_index] == 'logistic') and (x[size_index] == (10, 1))]
mlp_logistic_20_results_Waltz = [x[3] for x in results['mlp']['zip'] if (x[activation_index] == 'logistic') and (x[size_index] == (20, 1))]
mlp_logistic_30_results_Waltzt = [x[3] for x in results['mlp']['zip'] if (x[activation_index] == 'logistic') and (x[size_index] == (30, 1))]
mlp_logistic_50_results_Waltzt = [x[3] for x in results['mlp']['zip'] if (x[activation_index] == 'logistic') and (x[size_index] == (50, 1))]

mlp_tanh_10_results_Waltz = [x[3] for x in results['mlp']['zip'] if (x[activation_index] == 'tanh') and (x[size_index] == (10, 1))]
mlp_tanh_20_results_Waltz = [x[3] for x in results['mlp']['zip'] if (x[activation_index] == 'tanh') and (x[size_index] == (20, 1))]
mlp_tanh_30_results_Waltz = [x[3] for x in results['mlp']['zip'] if (x[activation_index] == 'tanh') and (x[size_index] == (30, 1))]
mlp_tanh_50_results_Waltz = [x[3] for x in results['mlp']['zip'] if (x[activation_index] == 'tanh') and (x[size_index] == (50, 1))]

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import matplotlib.patches as patches
from matplotlib import pylab
from matplotlib.ticker import FormatStrFormatter

mpl.rcParams['text.usetex'] = True
mpl.rcParams['font.family'] = 'serif'
mpl.rcParams['font.serif'] = 'cm'

fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, sharex = False, sharey = False, figsize = (12, 8))

# Select top left subplot.
ax1 = plt.subplot(2, 2, 1)

ax1.semilogx(svm_linear_C, svm_linear_results, color = 'red')

ax1.set_title('Chart A', fontsize = 16)
ax1.set_xlabel(r'$\log(C)', fontsize = 16)
ax1.set_ylabel('Mean Area Under ROC Curve', fontsize = 16)
ax1.yaxis.set_major_formatter(FormatStrFormatter('%.2f'))

ax1.set_ylim(0.35, 0.85)

linear_line = mlines.Line2D([], [], linestyle = '-', color = 'red', label = 'Linear kernel')
plt.legend(handles = [linear_line], loc = 'lower center', frameon = False)

# Select right subplot.
ax2 = plt.subplot(2, 2, 2)

# Add lines.
ax2.semilogx(forest_n_estimators, forest_max_depth_1_results, color = 'red')
ax2.semilogx(forest_n_estimators, forest_max_depth_10_results, color = 'blue')
ax2.semilogx(forest_n_estimators, forest_max_depth_100_results, color = 'black')
ax2.semilogx(forest_n_estimators, forest_max_depth_1000_results, color = 'green')

ax2.set_title('Chart B', fontsize = 16)
ax2.set_xlabel(r'$\log($size of forest$)', fontsize = 16)
ax2.set_ylabel('Mean Area Under ROC Curve', fontsize = 16)
ax2.yaxis.set_major_formatter(FormatStrFormatter('%.2f'))

ax2.set_ylim(0.35, 0.85)
             
forest_1 = mlines.Line2D([], [], linestyle = '-', color = 'red', label = r'Max depth $= 10^0$')
forest_10 = mlines.Line2D([], [], linestyle = '-', color = 'blue', label = r'Max depth $= 10^1$')
forest_100 = mlines.Line2D([], [], linestyle = '-', color = 'black', label = r'Max depth $= 10^2$')
forest_1000 = mlines.Line2D([], [], linestyle = '-', color = 'green', label = r'Max depth $= 10^3$')
plt.legend(handles = [forest_1, forest_10, forest_100, forest_1000], loc = 'lower center', frameon = False)

# Select bottom left subplot.
ax3 = plt.subplot(2, 2, 3)

ax3.semilogx(logistic_C, logistic_results, color = 'red')

ax3.set_title('Chart A', fontsize = 16)
ax3.set_xlabel(r'$\log(C)$', fontsize = 16)
ax3.set_ylabel('Mean Area Under ROC Curve', fontsize = 16)
ax3.yaxis.set_major_formatter(FormatStrFormatter('%.2f'))

ax3.set_ylim(0.35, 0.85)

# Select right subplot.
ax4 = plt.subplot(2, 2, 4)

ax4.semilogx(mlp_learning_rate_init, mlp_identity_10_results, color = 'red', linestyle = '-')
ax4.semilogx(mlp_learning_rate_init, mlp_logistic_10_results, color = 'red', linestyle = '--')
ax4.semilogx(mlp_learning_rate_init, mlp_tanh_10_results, color = 'red', linestyle = 'dotted')

ax4.semilogx(mlp_learning_rate_init, mlp_identity_20_results, color = 'blue', linestyle = '-')
ax4.semilogx(mlp_learning_rate_init, mlp_logistic_20_results, color = 'blue', linestyle = '--')
ax4.semilogx(mlp_learning_rate_init, mlp_tanh_20_results, color = 'blue', linestyle = 'dotted')

ax4.semilogx(mlp_learning_rate_init, mlp_identity_30_results, color = 'black', linestyle = '-')
ax4.semilogx(mlp_learning_rate_init, mlp_logistic_30_results, color = 'black', linestyle = '--')
ax4.semilogx(mlp_learning_rate_init, mlp_tanh_30_results, color = 'black', linestyle = 'dotted')

ax4.semilogx(mlp_learning_rate_init, mlp_identity_50_results, color = 'green', linestyle = '-')
ax4.semilogx(mlp_learning_rate_init, mlp_logistic_50_results, color = 'green', linestyle = '--')
ax4.semilogx(mlp_learning_rate_init, mlp_tanh_50_results, color = 'green', linestyle = 'dotted')

ax4.set_title('Chart B', fontsize = 16)
ax4.set_xlabel(r'$\log($learning rate$)', fontsize = 16)
ax4.set_ylabel('Mean Area Under ROC Curve', fontsize = 16)
ax4.yaxis.set_major_formatter(FormatStrFormatter('%.2f'))

ax4.set_ylim(0.35, 0.85)

neurons_10 = patches.Patch(color = 'red', label = 'Hidden layer size = $10$')
neurons_20 = patches.Patch(color = 'blue', label = 'Hidden layer size = $20$')
neurons_30 = patches.Patch(color = 'black', label = 'Hidden layer size = $30$')
neurons_50 = patches.Patch(color = 'green', label = 'Hidden layer size = $50$')

identity_function = mlines.Line2D([], [], linestyle = '-', color = 'gray', label = r'Identity function')
sigmoid_function = mlines.Line2D([], [], linestyle = '--', color = 'gray', label = r'Sigmoid function')
tanh_function = mlines.Line2D([], [], linestyle = 'dotted', color = 'gray', label = r'Tanh function')

handles = [identity_function, sigmoid_function, tanh_function, neurons_10, neurons_20, neurons_30, neurons_50]

plt.legend(handles = handles, loc = 'lower center', frameon = False)


plt.tight_layout()
plt.show()
fig.savefig('GridSearchAllMethodsWaltzFeatures.png', dpi = 1000)

In [ ]:
# print out best hyperparameter settings
for method in methods:
    print('Best', method, 'hyperparameters:', classifiers[method].best_estimator_)

In [ ]:
# apply fitted linear svm classifier with best hyperparameters to test data
predictions = {}
predicted_pos = {}
for method in ['forest', 'logistic', 'mlp']:
    predictions.update({method : classifiers[method].best_estimator_.predict_proba(X_test)})
    predicted_pos[method] = []
    for pair in predictions[method]:
        predicted_pos[method].append(pair[1])

In [ ]:
predicted_pos['forest']

In [ ]:
# print out highest mean AUC values
for method in methods:
    print('Highest', method, 'mean AUC:', max(results[method]['mean_test_score']))

In [ ]:
# import ROC curve package
from sklearn.metrics import roc_curve

true_pos_rates = {}
false_pos_rates = {}
thresholds = {}

for method in methods:
    (false_pos_rates[method], true_pos_rates[method], thresholds[method]) = roc_curve(y_test, predicted_pos[method])

In [ ]:
# plot ROC curves
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, sharex = False, sharey = False, figsize = (12, 8))

# Select top left subplot.
ax1 = plt.subplot(2, 2, 1)

ax1.plot(false_pos_rates['svm'].tolist(), true_pos_rates['svm'].tolist(), color = 'blue')
ax1.set_title('Chart A: Support Vector Machine', fontsize = 16)
ax1.set_xlabel('False Positive Rate', fontsize = 16)
ax1.set_ylabel('True Positive Rate', fontsize = 16)

ax1.set_xlim(-0.05, 1.05)
ax1.set_ylim(-0.05, 1.05)

ax1.plot(np.linspace(0, 1), np.linspace(0, 1), linestyle = '--', color = 'red')

ROC_curve = mlines.Line2D([], [], linestyle = '-', color = 'blue', label = r'Linear kernel: $C = ' + str(classifiers['svm'].best_estimator_.C) + '$')
rand_chance = mlines.Line2D([], [], linestyle = '--', color = 'red', label = r'Random chance')
plt.legend(handles = [ROC_curve, rand_chance], loc = 'lower right', frameon = False)

# Select top right subplot.
ax2 = plt.subplot(2, 2, 2)

ax2.plot(false_pos_rates['forest'].tolist(), true_pos_rates['forest'].tolist(), color = 'blue')
ax2.set_title('Chart B: Random Forest', fontsize = 16)
ax2.set_xlabel('False Positive Rate', fontsize = 16)
ax2.set_ylabel('True Positive Rate', fontsize = 16)

ax2.set_xlim(-0.05, 1.05)
ax2.set_ylim(-0.05, 1.05)

ax2.plot(np.linspace(0, 1), np.linspace(0, 1), linestyle = '--', color = 'red')

ROC_curve = mlines.Line2D([], [], linestyle = '-', color = 'blue', label = r'' + str(classifiers['forest'].best_estimator_.n_estimators) + ' trees, max depth $= ' + str(classifiers['forest'].best_estimator_.max_depth) + '$' )
rand_chance = mlines.Line2D([], [], linestyle = '--', color = 'red', label = r'Random chance')
plt.legend(handles = [ROC_curve, rand_chance], loc = 'lower right', frameon = False)

# Select bottom left subplot.
ax3 = plt.subplot(2, 2, 3)

ax3.plot(false_pos_rates['logistic'].tolist(), true_pos_rates['logistic'].tolist(), color = 'blue')
ax3.set_title('Chart C: Logistic Regression', fontsize = 16)
ax3.set_xlabel('False Positive Rate', fontsize = 16)
ax3.set_ylabel('True Positive Rate', fontsize = 16)

ax3.set_xlim(-0.05, 1.05)
ax3.set_ylim(-0.05, 1.05)

ax3.plot(np.linspace(0, 1), np.linspace(0, 1), linestyle = '--', color = 'red')

ROC_curve = mlines.Line2D([], [], linestyle = '-', color = 'blue', label = r'Logistic regression: C $= ' + str(classifiers['logistic'].best_estimator_.C) + '$')
rand_chance = mlines.Line2D([], [], linestyle = '--', color = 'red', label = r'Random chance')
plt.legend(handles = [ROC_curve, rand_chance], loc = 'lower right', frameon = False)

# Select bottom right subplot.
ax4 = plt.subplot(2, 2, 4)

ax4.plot(false_pos_rates['mlp'].tolist(), true_pos_rates['mlp'].tolist(), color = 'blue')
ax4.set_title('Chart D: Multilayer Perceptron', fontsize = 16)
ax4.set_xlabel('False Positive Rate', fontsize = 16)
ax4.set_ylabel('True Positive Rate', fontsize = 16)

ax4.set_xlim(-0.05, 1.05)
ax4.set_ylim(-0.05, 1.05)

ax4.plot(np.linspace(0, 1), np.linspace(0, 1), linestyle = '--', color = 'red')

ROC_curve = mlines.Line2D([], [], linestyle = '-', color = 'blue', label = r'Activation = ' + str(classifiers['mlp'].best_estimator_.activation) + r', ' + str(classifiers['mlp'].best_estimator_.hidden_layer_sizes[0]) + r' hidden neurons')
rand_chance = mlines.Line2D([], [], linestyle = '--', color = 'red', label = r'Random chance')
plt.legend(handles = [ROC_curve, rand_chance], loc = 'lower right', frameon = False)

plt.tight_layout()
plt.show()
fig.savefig('ROCCurvesSequenceBasedWaltzFeatures.png', dpi = 1000)

In [ ]:
ROC_lines = {}
best_perf = {}
best_dist = {}
best_index = {}
best_thresholds = {}

for method in methods:
    ROC_lines[method] = list(zip(false_pos_rates[method], true_pos_rates[method]))
    best_perf[method] = (1, 0)
    best_dist[method] = 1
    best_index[method] = 0
    for i in range(0, len(ROC_lines[method])):
        new_dist = np.sqrt((0 - ROC_lines[method][i][0]) + (1 - ROC_lines[method][i][1]))
        if (new_dist < best_dist[method]):
            best_perf[method] = ROC_lines[method][i]
            best_index[method] = i
    # find best probability threshold
    best_thresholds[method] = thresholds[method][best_index[method]]
    print(method, 'best probability threshold =', best_thresholds[method])

In [ ]:
binary_predictions = {}

for method in methods:
    binary_predictions[method] = []
    for probability in predicted_pos[method]:
        if (probability >= best_thresholds[method]):
            binary_predictions[method].append(1)
        else:
            binary_predictions[method].append(0)

In [ ]:
# print number of amyloids in test set
y_test.sum()

In [ ]:
# import package for calculating confusion matrices
from sklearn.metrics import confusion_matrix

# print out confusion matrix for each machine learning method
for method in methods:
    print(method, 'confusion matrix =', confusion_matrix(y_test, binary_predictions[method]))

In [ ]:
# import package for calculating accuracy
from sklearn.metrics import accuracy_score

# print out accuracies for each method
for method in methods:
    print(method, 'accuracy =', accuracy_score(y_test, binary_predictions[method]))

In [ ]:
# import package for calculating Matthews correlation coefficient
from sklearn.metrics import matthews_corrcoef

# print Matthews correlation coefficients
for method in methods:
    print(method, 'MCC =', matthews_corrcoef(y_test, binary_predictions[method]))

In [ ]:
# import pickling library to save trained machine learning classifiers to file
import pickle

fileObject = open('Pickle_Classifiers_Waltz_Features','wb') 
pickle.dump(classifiers, fileObject)   
fileObject.close()

fileObject = open('Pickle_Results_Waltz_Features','wb') 
pickle.dump(results, fileObject)   
fileObject.close()

fileObject = open('Pickle_X_test_Waltz_Features','wb') 
pickle.dump(X_test, fileObject)   
fileObject.close()

fileObject = open('Pickle_y_test_Waltz_Features','wb') 
pickle.dump(y_test, fileObject)   
fileObject.close()

In [ ]:
# import pickling library to load trained machine learning classifiers
#import pickle

#fileObject = open('Pickle_Classifiers_Best_Features', 'rb') 
#classifiers = pickle.load(fileObject)

#fileObject = open('Pickle_Results_Best_Features', 'rb') 
#results = pickle.load(fileObject)  

#fileObject = open('Pickle_X_test_Best_Features', 'rb') 
#X_test = pickle.load(fileObject)